In [1]:
import netgen.gui
from ngsolve import *
from netgen.geom2d import SplineGeometry
from math import pi
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
geo = SplineGeometry()
geo.AddRectangle((-1,-1),(1,1),bc=1)
mesh = Mesh( geo.GenerateMesh(maxh=0.5))

V = H1(mesh, order=5, dirichlet=[1])
# finite element space and gridfunction to represent the heatflux:
space_flux = HDiv(mesh, order=2)
gf_flux = GridFunction(space_flux, "flux")

u = V.TrialFunction()
v = V.TestFunction()

r = sqrt(x*x+y*y)
r2 = x*x+y*y
R = 1/2

alpha = IfPos(r-R,2*log(1/R),1)

solution = IfPos(r-R,1.0/16.0*log(r2)/log(R*R),1.0/8.0-r2/4.0)
sol_flux = alpha * IfPos(r-R,1.0/16.0/log(R*R)*
                     CoefficientFunction((2*x/r2,2*y/r2)),
                     CoefficientFunction((-x/2.0,-y/2.0)))
rhs = IfPos(r-R,0,1)

In [ ]:
a = BilinearForm(V, symmetric=False)
a += SymbolicBFI(alpha*grad(u)*grad(v))

f = LinearForm(V)
f += SymbolicLFI(rhs*v)

gfu = GridFunction(V)
flux = alpha * grad(gfu)
#gf_flux.Set(flux)
Draw (gfu,mesh,"u")
Draw (grad(gfu),mesh,"grad_u")

In [ ]:
def SolveBVP():
    V.Update()
    gfu.Update()
    
    space_flux.Update()
    gf_flux.Update()
    
    a.Assemble()
    f.Assemble()
    gfu.Set(solution)
    f.vec.data -= a.mat * gfu.vec
    gfu.vec.data += a.mat.Inverse(V.FreeDofs(),"umfpack") * f.vec
    Redraw (blocking=True)

In [ ]:
def CalcError():
    space_flux.Update()
    gf_flux.Update()
    # interpolate finite element flux into H(div) space:
    gf_flux.Set (flux)
    # compute error on every element:
    err = 1/alpha*(flux - sol_flux)*(flux - sol_flux)
    elerr = Integrate (err, mesh, VOL, element_wise=True)
    
    # sort elements (corresponding to error contribution)
    err_and_el_sorted = sorted([(entry,i) for i, entry in enumerate(elerr)], key= lambda x:-x[0])
    # reset marks
    marks = [False for el in mesh.Elements()]

    # mark element with largest error until 10% of the error is on marked elements:
    sumerr = sum(elerr)    
    accsum = 0
    for err,el in err_and_el_sorted:
        if accsum < 0.1 * sumerr:
            marks[el] = True
            accsum += err
        else:
            break
    
    print ("V.ndof = ", V.ndof)
    H1error = sqrt(Integrate (1/alpha*(flux - sol_flux)*(flux - sol_flux), mesh, VOL))
    print ("weighted H1 (semi norm) error = ", H1error)
    L2error = sqrt(Integrate ((gfu - solution)*(gfu - solution), mesh, VOL))
    print ("hier L2 error = ", L2error)
    
        # call the refinement according to the marks:
    for el in mesh.Elements():
        mesh.SetRefinementFlag(el, marks[el.nr])
    return  L2error,V.ndof,sumerr

In [ ]:
with TaskManager():
    L2error_2 = []
    H1error_2 = []
    d_2 = []
    #L2error_2 = []
    #d_2 = []
    while V.ndof < 10000:  
        SolveBVP()
        temp,d,H1flux = CalcError()
        L2error_2.append(temp)
        H1error_2.append(H1flux)
        d_2.append(d)
        mesh.Refine()
    SolveBVP()

### Constant estimation for the second point

In [ ]:
C_ap = 0
C_un = 0
#C = 0
for i in range(len(L2error_2)):
    C_ap += L2error_2[i]+H1error_2[i]
C_ap = C_ap/len(L2error_2)

for i in range(len(L2error_1)):
    C_un += L2error_1[i]+H1error_1[i]
C_un = C_un/len(L2error_1)

In [ ]:
print("C uniformt: ",C_un,"C adapt: ",C_ap)

In [ ]:
plt.figure()
plt.title("uniform refinements vs adaptive refinements")
plt.grid()
plt.semilogy(d_2,L2error_2,'b',label='L2_error uniform refinements')
plt.semilogy(d_2,H1error_2,'g',label='L2_error adaptive refinements')
#plt.semilogy(d_1,H1error_1)
#plt.semilogy(d_2,H1error_2)
plt.xlabel("dim V_h")
plt.ylabel("L2_norm")
plt.legend()
#plt.savefig('uniform refinements vs adaptive refinements')

### explenation
The blue line represents the error with the uniform refinments. That means the mesh is uniformt over the hole domain. The other line th error from the adaptive one. The algorytmus measures the error and define the new grid in the "critical area". It lies on the boundary on the circuit with an radius of $R$.

In [ ]:
plt.figure()
plt.title("gradient recovery error vs L2_error (adaptive)")
plt.grid()
#plt.semilogy(d_1,L2error_1)
plt.semilogy(d_2,L2error_2,'g',label='L2_error adaptive refinements')
plt.semilogy(d_2,H1error_2,color='orange',label='GR_error adaptive refinements')
plt.xlabel("dim V_h")
plt.ylabel("error")
plt.legend()
#plt.savefig('gradient recovery error vs L2_error (adaptive)')

In [ ]:
plt.figure()
plt.title("gradient recovery error vs L2_error (uniform)")
plt.grid()
#plt.semilogy(d_1,L2error_1)
plt.semilogy(d_1,L2error_1,'b',label='L2_error uniform refinements')
plt.semilogy(d_1,H1error_1,color='red',label='GR_error uniform refinements')
plt.xlabel("dim V_h")
plt.ylabel("error")
plt.legend()
#plt.savefig('gradient recovery error vs L2_error (uniform)')